In [7]:
import os
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = '/Users/andreanavarrete/workspace/columbia/courses/capstone/BlackLivesMatter/dsi-capstone-f19-group-1-51b3e5d95350.json'


In [151]:
import json
import matplotlib.pyplot as plt
import re
import swifter

from google.cloud import bigquery

%load_ext google.cloud.bigquery
plt.style.use('fivethirtyeight')

The google.cloud.bigquery extension is already loaded. To reload it, use:
  %reload_ext google.cloud.bigquery


# Read tweets

In [9]:
bigquery_client = bigquery.Client(project='dsi-capstone-f19-group-1')

In [11]:
%%bigquery tweets
select id, id_str, full_text
from tweets.all_valid_tweets limit 50;

In [14]:
tweets.head()

,id,id_str,full_text
0,501956853446635500,501956853446635520,RT @michaelcalhoun: It gets crazy. #ferguson h...
1,500859636279681000,500859636279681024,RT @lowKeyFamuz: Mike Brown Trayvon Martin Sam...
2,498311125998395400,498311125998395392,RT @grimcity: Mayor of #Ferguson is James Know...
3,498648533298192400,498648533298192386,RT @MichaelSkolnik: Tension rising at candleli...
4,500556360178991100,500556360178991105,"""Michael Brown's Shooting May Have Been Live-T..."


## Read LWIC json

In [178]:
liwc_filename = '../../LIWC2015_changed.json'
with open(liwc_filename) as json_file:
    liwc = json.load(json_file)

In [179]:
liwc.keys()

dict_keys(['Function', 'Pronoun', 'Ppron', 'I', 'We', 'You', 'SheHe', 'They', 'Ipron', 'Article', 'Prep', 'Auxverb', 'Adverb', 'Conj', 'Negate', 'Verb', 'Adj', 'Compare', 'Interrog', 'Number', 'Quant', 'Affect', 'Posemo', 'Negemo', 'Anx', 'Anger', 'Sad', 'Social', 'Family', 'Friend', 'Female', 'Male', 'CogProc', 'Insight', 'Cause', 'Discrep', 'Tentat', 'Certain', 'Differ', 'Percept', 'See', 'Hear', 'Feel', 'Bio', 'Body', 'Health', 'Sexual', 'Ingest', 'Drives', 'Affiliation', 'Achieve', 'Power', 'Reward', 'Risk', 'FocusPast', 'FocusPresent', 'FocusFuture', 'Relativ', 'Motion', 'Space', 'Time', 'Work', 'Leisure', 'Home', 'Money', 'Relig', 'Death', 'Informal', 'Swear', 'Netspeak', 'Assent', 'Nonflu', 'Filler'])

In [186]:
def get_keywords(key, text):
    new_vals = ['{}\w*'.format(x.lower().replace('*', ''))  if '*' in x else x.lower()
                for x in liwc[key]]
    match_list = []
    for w in new_vals:
        for tw in text.split(' '):
            match = re.findall(r'\b{}\b'.format(w), tw.lower())
            if len(match) > 0:
                match_list += match
    return match_list

def anger(text):
    return get_keywords('Anger', text)

In [187]:
keys = ['Anger', 'Sexual',
        'Anx', 'Sad', 'Social', 'Percept', 
        'Drives', 'Affiliation',
        'Achieve', 'Power', 'Reward', 'Risk', 'Death']

for key in keys:
    tweets['{}_words'.format(key)] = tweets['full_text'].swifter.apply(lambda x: get_keywords(key, x))

In [188]:
tweets

,id,id_str,full_text,Anger_words,Sexual_words,Anx_words,Sad_words,Social_words,Percept_words,Drives_words,Affiliation_words,Achieve_words,Power_words,Reward_words,Risk_words,Death_words
0,501956853446635500,501956853446635520,RT @michaelcalhoun: It gets crazy. #ferguson h...,[],[],[],[],[rt],[],[gets],[],[],[],[gets],[],[]
1,500859636279681000,500859636279681024,RT @lowKeyFamuz: Mike Brown Trayvon Martin Sam...,[],[],[],[],[rt],[brown],[up],[],[],[up],[],[],[]
2,498311125998395400,498311125998395392,RT @grimcity: Mayor of #Ferguson is James Know...,[],[],[],[],"[email, phone, rt]",[phone],[],[],[],[],[],[],[]
3,498648533298192400,498648533298192386,RT @MichaelSkolnik: Tension rising at candleli...,[],[],[tension],[],[rt],"[candlelight, photo]",[police],[],[],[police],[],[],[]
4,500556360178991100,500556360178991105,"""Michael Brown's Shooting May Have Been Live-T...",[],[],[],[],"[citizen, tweeted]",[brown],"[tweeted, vpm8ojfnni]",[tweeted],[],[vpm8ojfnni],[],[],[]
5,502517923324624900,502517923324624896,RT @FergusonUnity: We're also covering the off...,[],[],[],[],"[crowds, rt, we, we're, who]",[],"[officer, we, we're]","[we, we're]",[],[officer],[],[],[]
6,501593709063192600,501593709063192577,"RT @MoreAndAgain: Meanwhile, in NYC the cop wh...",[killed],[],[],[],"[rt, who, whose]",[],[cop],[],[],[cop],[],[],"[death, killed]"
7,501969759341396000,501969759341395968,"RT @McBlondeLand: Reporter says ""OMG I haven't...",[],[],[],[],"[rt, says]","[says, seen]",[],[],[],[],[],[],[]
8,500050920852709400,500050920852709376,"This all could've been avoided if, say, a cop ...",[],[],[avoided],[],"[kid, say]",[say],"[avoided, cop, kid, refrained]",[],[],"[cop, kid]",[],"[avoided, refrained]",[]
9,499531314794864640,499531314794864640,RT @TheAnonMessage: BREAKING: Instead of dispe...,[protesters],[],[],[],"[rt, their, their]",[hands],"[got, up]",[],[],[up],[got],[],[]
